In [21]:
import time
import odbc
import pandas as pd

In [2]:
import re
def onlyKorean(s):
    hangeul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    hangeul_s = hangeul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    not_hangeul_l = hangeul.findall(s) # 정규식에 일치되는 부분을 리스트 형태로 저장
    return hangeul_s,not_hangeul_l

In [3]:
connect=odbc.odbc('tv')
db=connect.cursor()

In [27]:
db.execute("SELECT d.drama_id,d.title FROM (select ori.drama_id,ori.video_id from tv.tvcast_video_list as ori,(SELECT drama_id,count(*) cnt FROM tv.tvcast_video_list group by drama_id having cnt>=98 and cnt<=595) as re where ori.drama_id=re.drama_id) AS v, tv.tvcast_drama_list AS d WHERE d.drama_id = v.drama_id AND d.title NOT LIKE '%웹%' group by d.drama_id;")
naver = db.fetchall()

In [28]:
db.execute("SELECT * FROM tv.drama;")
rate = db.fetchall()

In [29]:
data=[]
temp=[]
for rate_row in rate:
    rate_title = rate_row[2]
    for naver_row in naver:
        naver_title=naver_row[1]
        naver_title_re,sp_chars = onlyKorean(naver_title)
        naver_title_re = naver_title_re.strip()
        if len(naver_title_re)==0:
            continue
        if naver_title_re in rate_title:
            temp=[rate_row[0],rate_row[2],naver_row[0],naver_row[1]]
            data.append(temp)

In [30]:
df = pd.DataFrame(data)

In [31]:
df.head()

,0,1,2,3
0,92,주말극특별기획(KBS신화는계속된다엄마가뿔났다),556,엄마
1,94,닥터스,350,닥터스
2,102,주말연속극(엄마가뿔났다),556,엄마
3,115,설날특집대박식당의비밀,411,대박
4,169,주말연속극(엄마가뿔났다<재>),556,엄마


In [26]:
writer = pd.ExcelWriter('master_table.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()